In [404]:
import pandas as pd
import datetime
import numpy as np
import sklearn
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import *
from sklearn.neural_network import MLPRegressor
#from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
df_train=pd.read_csv("C:/Users/Akash/Documents/ADS/Assignment2/Appliances-energy-prediction-data-master/training.csv")
df_train['date']=pd.to_datetime(df_train['date'])
#df_train['year']=df_train['date'].dt.year
df_train['month']=df_train['date'].dt.month
df_train['day']=df_train['date'].dt.day
df_train['time_hr_24']=df_train['date'].dt.hour
df_train['time_min']=df_train['date'].dt.minute
morning=range(6,12)
afternoon=range(12,17)
evening=range(17,22)
def time_slot(x):
    if x in morning:
        return 'morning'
    elif x in afternoon:
        return 'afternoon'
    elif x in evening:
        return 'evening'
    else:
        return 'night'
#df_train['time_slot']=df_train['time_hr_24'].map(time_slot)
df_train.drop(['date'],axis=1,inplace=True)
df_train=pd.get_dummies(df_train,prefix=['WS','DOW'],columns=['WeekStatus','Day_of_week'])
df_train.shape

(14803, 42)

In [405]:
#for training dataset
x_train=df_train.iloc[:,1:]
y_train=df_train['Appliances']
#col_list=['NSM','RH_1','RH_2','T3','RH_3','Press_mm_hg','TS_night','T1','T2','T4','RH_4','T5','RH_5','T6','RH_6',
#                      'T7','RH_7','T8','RH_8','RH_9','T_out','RH_out','Windspeed','Tdewpoint']
#x_train=x_train[col_list]
scaler.fit(x_train)
x_train=scaler.transform(x_train)
#x_train.head()

In [546]:
mlp = MLPRegressor(hidden_layer_sizes=(280,280,280),max_iter=500)
mlp.fit(x_train,y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(280, 280, 280), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [547]:
y_train_pred=mlp.predict(x_train)
print("R2   :",r2_score(y_train,y_train_pred))
print("MAE  :",mean_absolute_error(y_train,y_train_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_train,y_train_pred)))
print("MAPE :",mean_absolute_percentage_error(y_train,y_train_pred))
#print(confusion_matrix(y_train,y_train_pred))

R2   : 0.419674949147
MAE  : 40.2604663324
RMSE : 78.3307815482
MAPE : 40.2324688137


In [548]:
#for testing dataset
df_test=pd.read_csv("C:/Users/Akash/Documents/ADS/Assignment2/Appliances-energy-prediction-data-master/testing.csv")
df_test['date']=pd.to_datetime(df_test['date'])
#df_test['year']=df_test['date'].dt.year
df_test['month']=df_test['date'].dt.month
df_test['day']=df_test['date'].dt.day
df_test['time_hr_24']=df_test['date'].dt.hour
df_test['time_min']=df_test['date'].dt.minute
#df_test['time_slot']=df_test['time_hr_24'].map(time_slot)
df_test.drop(['date'],axis=1,inplace=True)
#df_test.head()
df_test=pd.get_dummies(df_test,prefix=['WS','DOW'],columns=['WeekStatus','Day_of_week'])
#print(df_test.describe())

In [549]:
x_test=df_test.iloc[:,1:]
#x_test=x_test[col_list]
x_test=scaler.transform(x_test)
y_test=df_test['Appliances']
baseline_errors = abs(96.745742 - y_test)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))
#evaluate(rf,x_test,y_test)

Average baseline error:  59.08


In [550]:
y_test_pred=mlp.predict(x_test)
print("R2   :",r2_score(y_test,y_test_pred))
print("MAE  :",mean_absolute_error(y_test,y_test_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_test,y_test_pred)))
print("MAPE :",mean_absolute_percentage_error(y_test,y_test_pred))

R2   : 0.315196153964
MAE  : 42.2853378244
RMSE : 84.0800622356
MAPE : 41.9931921129


In [402]:
parameters={
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(50,50,50),(140,140,140), (210,210,210)],
'alpha': [10.0 ** -np.arange(1, 7)],
'solver' : ["lbfgs","sgd", "adam"],
'activation': [ "relu", "Tanh"]
}

In [403]:
mlp_grid=GridSearchCV(estimator=MLPRegressor,param_grid=parameters,n_jobs=-1,verbose=2,cv=3)
mlp_grid.fit(x_train, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


C:\Users\Akash\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


TypeError: get_params() missing 1 required positional argument: 'self'

In [400]:
y_train_pred=mlp_grid.predict(x_train)
print("R2   :",r2_score(y_train,y_train_pred))
print("MAE  :",mean_absolute_error(y_train,y_train_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_train,y_train_pred)))
print("MAPE :",mean_absolute_percentage_error(y_train,y_train_pred))

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [6]:
import pandas as pd
import datetime
import numpy as np
import sklearn
from sklearn.cross_validation import train_test_split 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model
from sklearn.metrics import *
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
weekend = ['Saturday','Sunday']
def week_day_type(x):
    if x in weekend:
        return 'weekends'
    else:
        return 'weekdays'
def time_slot(x):
    if x in morning:
        return 'morning'
    elif x in afternoon:
        return 'afternoon'
    elif x in evening:
        return 'evening'
    else:
        return 'night'  
df=pd.read_csv("C:/Users/Akash/Documents/ADS/Assignment2/Appliances-energy-prediction-data-master/energydata_complete.csv")
df['date']=pd.to_datetime(df['date'])
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df['day_of_week']=df['date'].dt.weekday_name
df['time_hr_24']=df['date'].dt.hour
df['time_min']=df['date'].dt.minute
df['week_day_type']=df['day_of_week'].map(week_day_type)
morning=range(6,12)
afternoon=range(12,17)
evening=range(17,22)  
df['time_slot']=df['time_hr_24'].map(time_slot)
df.drop(['date'],axis=1,inplace=True)
df=pd.get_dummies(df,prefix=['DOW','TS','WDT'],columns=['day_of_week','time_slot','week_day_type'])
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
print(df_train.shape)
print(df_test.shape)

(13814, 46)
(5921, 46)


In [7]:
x_train=df_train.iloc[:,1:]
#print(x_train.shape)
y_train=df_train['Appliances']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,1:]
y_test=df_test['Appliances']
x_test_sc=scaler.transform(x_test)

In [8]:
parameters={
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(50,50,50),(140,140,140),(210,210,210)],
'alpha': [1.00000000e-01,   1.00000000e-02,   1.00000000e-03, 1.00000000e-04,   1.00000000e-05,   1.00000000e-06],
'solver' : ["lbfgs","sgd", "adam"],
'activation': [ "relu", "Tanh"]
}
mlp_grid=GridSearchCV(estimator=MLPRegressor(),param_grid=parameters,n_jobs=-1,verbose=2,cv=3)
mlp_grid.fit(x_train_sc, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\Akash\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\Akash\Anaconda3\lib\runpy.py

In [4]:
10.0 ** -np.arange(1, 7)

array([  1.00000000e-01,   1.00000000e-02,   1.00000000e-03,
         1.00000000e-04,   1.00000000e-05,   1.00000000e-06])

In [11]:
x_train.dtypes

lights             int64
T1               float64
RH_1             float64
T2               float64
RH_2             float64
T3               float64
RH_3             float64
T4               float64
RH_4             float64
T5               float64
RH_5             float64
T6               float64
RH_6             float64
T7               float64
RH_7             float64
T8               float64
RH_8             float64
T9               float64
RH_9             float64
T_out            float64
Press_mm_hg      float64
RH_out           float64
Windspeed        float64
Visibility       float64
Tdewpoint        float64
rv1              float64
rv2              float64
year               int64
month              int64
day                int64
time_hr_24         int64
time_min           int64
DOW_Friday         uint8
DOW_Monday         uint8
DOW_Saturday       uint8
DOW_Sunday         uint8
DOW_Thursday       uint8
DOW_Tuesday        uint8
DOW_Wednesday      uint8
TS_afternoon       uint8
